In [33]:
from datetime import datetime as dt
import re
import pandas as pd
from xbbg import blp
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
from datetime import timedelta as td
import statsmodels.api as sm
import re
import matplotlib.image as image
from ipywidgets import interact, IntSlider, Checkbox, Dropdown, Output, HBox, VBox, interactive, interactive_output, ToggleButton,Text, Button, DatePicker, IntText, ToggleButtons, RadioButtons,SelectMultiple
from IPython.display import display, clear_output
import itertools
from scipy import stats
from scipy.optimize import minimize 
from scipy.special import ndtr


# Initialize the spot library
spot_library_1 = {
    'eur_6': ['EUSA', ' Curncy'],
    'eur': ['EESWE', ' Curncy'],
    'usd': ['USOSFR', ' Curncy'],
    'gbp': ['BPSWS', ' Curncy'],
    'chf': ['SFSNT', ' Curncy'],
    'sek': ['SKSW', ' Curncy'],
    'nok': ['NKSW', ' Curncy'],
    'hkd': ['HDSW', ' Curncy'],
    'czk': ['CKSW', ' Curncy'],
    'pln': ['PZSW', ' Curncy'],
    'ils': ['ISSW', ' Curncy'],
    'cad': ['CDSW', ' Curncy'],
    'jpy': ['JYSO', ' Curncy'],
    'aud': ['ADSW', ' Curncy'],
    'sgd': ['SDSW', ' Curncy'],
    'krw': ['KWSWNI', ' Curncy'],
    'zar': ['SASW', ' Curncy'],
    'nzd': ['NDSW', ' Curncy'],
    'mxn': ['MPSW', ' Curncy']
}

spot_library = {'eur_6':['EUSA', ' Curncy'],'eur': ['EESWE', ' Curncy'], 'usd': ['USOSFR', ' Curncy'], 
                'gbp': ['BPSWS', ' Curncy'],'chf': ['SFSNT', ' Curncy'],'sek': ['SKSW', ' Curncy'],
                 'nok': ['NKSW', ' Curncy'], 'hkd': ['HDSW', ' Curncy'],'czk': ['CKSW', ' Curncy'],
                 'pln': ['PZSW', ' Curncy'],'ils':['ISSW', ' Curncy'],  'cad':['CDSW', ' Curncy'], 
                 'jpy':['JYSO', ' Curncy'], 'aud': ['ADSW', ' Curncy'],'sgd':['SDSW', ' Curncy'],
                'krw': ['KWSWNI', ' Curncy'],
                'zar': ['SASW', ' Curncy'],
                'nzd': ['NDSW', ' Curncy'],
                'mxn': ['MPSW', ' Curncy']} 

forward_library = {'eur_6': ['EUSA', ' Curncy'], 
                 'eur': ['S0514FS ', ' BLC Curncy'], 
                 'usd': ['S0490FS ', ' BLC Curncy'], 
                 'gbp': ['S0141FS ', ' BLC Curncy'],
                 'chf': ['S0234FS ', ' BLC Curncy'],
                 'sek': ['SD0020FS ', ' BLC Curncy'],
                 'nok': ['SD0313FS ', ' BLC Curncy'],
                 'hkd': ['HDFS', ' Curncy'],
                 'czk': ['S0320FS ', ' BLC Curncy'],
                 'pln': ['S0323FS ', ' BLC Curncy'],
                 'ils': ['ISFS', ' Curncy'],
                 'cad': ['CDFS', ' Curncy'],
                 'jpy': ['S0195FS ', ' BLC Curncy'],
                 'aud': ['SD0302FS ', ' BLC Curncy'],
                'sgd': ['SDFS', ' Curncy'],
                'krw': ['S0205FS ', ' BLC Curncy'],
                'zar': ['SAFS', ' Curncy'],
                'nzd': ['SD0015FS ', ' BLC Curncy'],
                'mxn': ['SD0083FS ', ' BLC Curncy']} 

def spot_ticker(dex,tenor):
    if dex.lower() == 'mxn':
        y = int(tenor[:-1]) * 13
        num = f'{y//12}{chr(64+(y%12))}'
    else:
        num = tenor[:-1]
    return f'{spot_library[dex][0]}{num}{spot_library[dex][1]}'

def forward_ticker(dex,fwd):
    dex = dex.lower()
    fwd = fwd.lower()
    old = ['eur_6','hkd','ils','cad','sgd','zar']
    
    if cut(fwd)[0] == '0y':
        return spot_ticker(dex,cut(fwd)[1])
    elif dex == 'eur_6':
        F,T = f(cut(fwd)[0]),t(cut(fwd)[1])
        return f'{forward_library[dex][0]}{F}{T}{forward_library[dex][1]}'
    elif dex in old:
        F,T = t(cut(fwd)[0]),t(cut(fwd)[1])
        return f'{forward_library[dex][0]}{F}{T}{forward_library[dex][1]}'
    else:
        
        return f'{forward_library[dex][0]}{fwd.upper()}{forward_library[dex][1]}'

# Swap structure builders
def cut(some):
    x = 0
    y = some[0]
    while y.isdigit():
        y = some[x]
        x += 1
    return some[:x], some[x:]

def f(tenor):
    x = re.findall(r'\d+', tenor)[0]
    num = ''
    if 'm' in tenor.lower():
        if int(x) // 12 > 0:
            num += '1'
        num += chr(64 + (int(x) % 12))
    else:
        if len(x) == 1:
            num += '0'
        num += x
    return num

def t(tenor):
    x = re.findall(r'\d+', tenor)[0]
    num = ''
    if 'm' in tenor.lower():
        if int(x) // 12 > 0:
            num += '1'
        else:
            num += '0'
        num += chr(64 + (int(x) % 12))
    else:
        if len(x) == 1:
            num += '0'
        num += x
    return num

def spot_ticker1(dex, tenor):
    if dex.lower() == 'mxn':
        y = int(tenor[:-1]) * 13
        num = f'{y // 12}{chr(64 + (y % 12))}'
    else:
        num = tenor[:-1]
    return f'{spot_library_1[dex][0]}{num}{spot_library_1[dex][1]}'

def forward_ticker1(dex, fwd):
    dex = dex.lower()
    fwd = fwd.lower()
    old = ['eur_6', 'hkd', 'ils', 'cad', 'sgd', 'zar']

    if cut(fwd)[0] == '0y':
        return spot_ticker(dex, cut(fwd)[1])
    elif dex == 'eur_6':
        F, T = f(cut(fwd)[0]), t(cut(fwd)[1])
        return f'{forward_library[dex][0]}{F}{T}{forward_library[dex][1]}'
    elif dex in old:
        F, T = t(cut(fwd)[0]), t(cut(fwd)[1])
        return f'{forward_library[dex][0]}{F}{T}{forward_library[dex][1]}'
    else:
        return f'{forward_library[dex][0]}{fwd.upper()}{forward_library[dex][1]}'

def swap_structure1(dex, structure, start, end='today', bps=True):
    f_fly = structure.count('/') == 2
    f_crv = structure.count('/') == 1
    fly = structure.count('s') == 3
    crv = structure.count('s') == 2
    out = max(2 - sum([i.isalpha() for i in structure]), 0)

    if f_fly or f_crv:
        legs = [forward_ticker1(dex, i) for i in structure.split('/')]
    elif fly or crv:
        legs = [spot_ticker1(dex, i) for i in [i + 'Y' for i in structure.split('s') if i.isdigit()]]
    else:
        legs = forward_ticker1(dex, ('0Y' * out) + structure)

    df = blp.bdh(legs, flds='px_last', start_date=start, end_date=end).bfill() * (100 if bps else 1)
    
    if df.empty or df.shape[1] < 1:
        return None
    
    s = pd.DataFrame({})
    if f_fly or fly:
        x = (2 * df.iloc[:, 1]) - (df.iloc[:, 0] + df.iloc[:, 2])
    elif f_crv or crv:
        x = df.iloc[:, 1] - df.iloc[:, 0]
    else:
        x = df.iloc[:, 0]

    s[f'{dex.upper()} {structure}'] = x
    return s

# Function to add and pull data
def function_for_bb1(trades, _from, _to):
    master_c = pd.DataFrame({})
    for j in trades:
        h = swap_structure1(dex=j[0].lower(), structure=j[1].lower(), start=_from, end=_to)
        if h is None:
            continue
        if master_c.shape == (0, 0):
            master_c = h
        else:
            master_c = master_c.join(h, how='outer')
    return master_c

# Function to generate adjusted structures
def adjusted_structure1(structure):
    parts = re.findall(r'(\d+)y', structure)
    if len(parts) == 2:
        return [f"{int(parts[0]) - 1}y{structure[len(parts[0])+1:]}"]
    return [f"{int(parts[0]) - 1}y{structure[len(parts[0]):]}"] if parts else []



In [34]:
curr = ['eur', 'usd', 'nok', 'sek', 'jpy', 'cad', 'aud', 'gbp', 'chf', 'nzd']
tens = ['1y1y', '2y2y', '3y3y', '5y5y', '3y2y', '2y1y', '5y2y', '7y3y', '10y10y', '15y15y', '20y10y']

In [35]:
len1 = len(curr)-1
len2 = len(tens)

total = len1*len2

total

99

In [36]:
gen_from_1 = dt.strptime("01/01/2019", "%d/%m/%Y")
gen_from_2 = dt(dt.today().year - 4, dt.today().month, dt.today().day)
gen_to_1 = dt.today()
trades2 = []

for i in tens:
    for j in curr:
        trades2.append(tuple([j,i]))




In [37]:
df = pd.read_excel('CURRENT.xlsx', nrows=0)

# Get the column names
column_names = df.columns.tolist()

In [38]:
column_names

['Dates',
 'SD0490FS 2Y1Y BLC Curncy',
 'CDFS0201 BLC Curncy',
 'S0141FS 2Y1Y BLC Curncy',
 'S0514FS 2Y1Y BLC Curncy',
 'SD0302FS 2Y1Y BLC Curncy',
 'NDFS0201 BLC Curncy',
 'SKFS0201 BLC Curncy',
 'SD0490FS 2Y2Y BLC Curncy',
 'CDFS0202 BLC Curncy',
 'S0141FS 2Y2Y BLC Curncy',
 'S0514FS 2Y2Y BLC Curncy',
 'SD0302FS 2Y2Y BLC Curncy',
 'NDFS0202 BLC Curncy',
 'SKFS0202 BLC Curncy',
 'SD0490FS 3Y2Y BLC Curncy',
 'CDFS0302 BLC Curncy',
 'S0141FS 3Y2Y BLC Curncy',
 'S0514FS 3Y2Y BLC Curncy',
 'SD0302FS 3Y2Y BLC Curncy',
 'NDFS0302 BLC Curncy',
 'SKFS0302 BLC Curncy',
 'SD0490FS 5Y2Y BLC Curncy',
 'CDFS0502 BLC Curncy',
 'S0141FS 5Y2Y BLC Curncy',
 'S0514FS 5Y2Y BLC Curncy',
 'SD0302FS 5Y2Y BLC Curncy',
 'SD0015FS 5Y2Y BLC Curncy',
 'SKFS0502 BLC Curncy',
 'SD0490FS 7Y3Y BLC Curncy',
 'CDFS0703 BLC Curncy',
 'S0141FS 7Y3Y BLC Curncy',
 'S0514FS 7Y3Y BLC Curncy',
 'SD0302FS 7Y3Y BLC Curncy',
 'NDFS0703 BLC Curncy',
 'SKFS0703 BLC Curncy',
 'SD0490FS 5Y5Y BLC Curncy',
 'CDFS0505 BLC Curncy',
 '

In [39]:
column_names = column_names[1:]

In [40]:
len(column_names)

91

In [41]:

gen_from_1 = dt.strptime("01/01/2019", "%d/%m/%Y")
gen_from_2 = dt(dt.today().year - 4, dt.today().month, dt.today().day)
gen_to_1 = dt.today()

master = blp.bdh(column_names[0], flds='px_last', start_date=gen_from_2, end_date=gen_to_1)

for i in column_names[1:]:
    try:
        df = blp.bdh(i, flds='px_last', start_date=gen_from_2, end_date=gen_to_1)
        master = master.join(df, how='outer')
    except:
        print(i)
        #print(blp.bdh(i, flds='px_last', start_date=gen_from_2, end_date=gen_to_1))
        #print(master)

SD0490FS 2Y2Y BLC Curncy
GTNZD2Y Govt.1


In [42]:
master

,SD0490FS 2Y1Y BLC Curncy,CDFS0201 BLC Curncy,S0141FS 2Y1Y BLC Curncy,S0514FS 2Y1Y BLC Curncy,SD0302FS 2Y1Y BLC Curncy,NDFS0201 BLC Curncy,SKFS0201 BLC Curncy,CDFS0202 BLC Curncy,S0141FS 2Y2Y BLC Curncy,S0514FS 2Y2Y BLC Curncy,...,RV0001P 5Y BLC Curncy,RV0007P 5Y BLC Curncy,RV0006P 5Y BLC Curncy,RV0002P 5Y BLC Curncy,RV0004P 5Y BLC Curncy,RV0008P 5Y BLC Curncy,RV0005P 5Y BLC Curncy,GTAUD5Y Govt,GTNZD5Y Govt,RV0001P 7Y BLC Curncy
,px_last,px_last,px_last,px_last,px_last,px_last,px_last,px_last,px_last,px_last,...,px_last,px_last,px_last,px_last,px_last,px_last,px_last,px_last,px_last,px_last
2020-08-31,0.0376,0.7324,-0.0510,-0.59120,0.3210,0.1024,0.0891,0.8372,0.0089,-0.57159,...,0.2851,0.3682,-0.0052,-0.6415,-0.4929,-0.1639,0.5377,0.4195,0.142,0.4952
2020-09-01,0.0265,0.7137,-0.0660,-0.60868,0.3006,0.0655,0.0944,0.8069,-0.0180,-0.59339,...,0.2694,0.3325,-0.0327,-0.6680,-0.5213,-0.1807,0.4831,0.4140,0.123,0.4667
2020-09-02,0.0416,0.7121,-0.0858,-0.64262,0.2873,0.0804,0.0552,0.7987,-0.0465,-0.62553,...,0.2663,0.3248,-0.0676,-0.7097,-0.5699,-0.2295,0.4377,0.4070,0.130,0.4525
2020-09-03,0.0294,0.7158,-0.0861,-0.64406,0.2902,0.0825,0.0234,0.8019,-0.0404,-0.63423,...,0.2621,0.3227,-0.0538,-0.7210,-0.5763,-0.2418,0.4399,0.3910,0.124,0.4399
2020-09-04,0.0604,0.7481,-0.0604,-0.63916,0.2652,0.0650,0.0289,0.8367,-0.0120,-0.62270,...,0.3140,0.3636,-0.0325,-0.7043,-0.5603,-0.2163,0.4794,0.3835,0.077,0.5072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-26,3.0575,NaN,3.5003,2.07379,3.4020,3.2077,1.9351,NaN,3.4426,2.10360,...,3.6886,2.8997,3.7188,2.1277,2.6002,2.6042,3.0404,3.4948,3.778,3.7449
2024-08-27,3.0404,NaN,3.5382,2.09027,3.4545,3.2671,1.9681,NaN,3.4920,2.12838,...,3.6731,2.8981,3.7736,2.1522,2.6404,2.6448,NaN,3.5543,3.826,3.7427
2024-08-28,3.0490,NaN,3.5483,2.07614,3.5330,3.2691,1.9479,NaN,3.5034,2.11321,...,3.6856,2.9244,3.7743,2.1305,2.6167,2.6266,3.0786,3.5845,3.843,3.7545


In [43]:
master.to_csv('xkmt_orig.csv')